## Part 1 - Load Data from Local with Pre-processing

In [34]:
import pandas as pd

In [35]:
# read .tsv file from local
df = pd.read_csv('/Users/voronica/Desktop/APAN5450/amazon_reviews_us_Digital_Video_Download_v1_00.tsv', 
                 sep='\t', header=0, on_bad_lines='skip', low_memory=False)

In [36]:
# display head of df to check data availabiltiy
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12190288,R3FU16928EP5TC,B00AYB1482,668895143,Enlightened: Season 1,Digital_Video_Download,5,0,0,N,Y,I loved it and I wish there was a season 3,I loved it and I wish there was a season 3... ...,2015-08-31
1,US,30549954,R1IZHHS1MH3AQ4,B00KQD28OM,246219280,Vicious,Digital_Video_Download,5,0,0,N,Y,As always it seems that the best shows come fr...,As always it seems that the best shows come fr...,2015-08-31
2,US,52895410,R52R85WC6TIAH,B01489L5LQ,534732318,After Words,Digital_Video_Download,4,17,18,N,Y,Charming movie,"This movie isn't perfect, but it gets a lot of...",2015-08-31
3,US,27072354,R7HOOYTVIB0DS,B008LOVIIK,239012694,Masterpiece: Inspector Lewis Season 5,Digital_Video_Download,5,0,0,N,Y,Five Stars,excellant this is what tv should be,2015-08-31
4,US,26939022,R1XQ2N5CDOZGNX,B0094LZMT0,535858974,On The Waterfront,Digital_Video_Download,5,0,0,N,Y,Brilliant film from beginning to end,Brilliant film from beginning to end. All of t...,2015-08-31


In [22]:
# The duplicate value checking indicates in the dataset,
# the same customer can have multiple reviews on different products,
# and the same product can receive multiple reviews from different customers
print(df.customer_id.duplicated().sum())
print(df.product_id.duplicated().sum())

1980190
3832311


In [23]:
# The duplicate values checking indicates in the dataset,
# The same customer only has only one review on the same product
duplicate_count = len(df[df.duplicated(['customer_id', 'review_id'])])
print(duplicate_count)

0


## Part 2 - Store Dataset into MongoDB

In [1]:
# Install PyMongo
!pip3 install -U pymongo


[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Initiate MongoDB Client Connection
from pymongo import MongoClient
client = MongoClient('localhost',27017)
db = client.apan5400
collection = db.amazon_reviews

In [ ]:
collection.drop()

In [42]:
# Insert data into Document
collection.insert_many(df.to_dict('records'))

In [43]:
# Remove data entries with null values

import numpy as np
# define the filter to remove documents with any null values
filter_query = {'$or': [{'review_headline': np.nan}, {'review_body': np.nan}, {'review_date': np.nan}]}

# delete documents that match the filter
result = collection.delete_many(filter_query)

# print the number of documents deleted
print(result.deleted_count, "documents deleted.")

241 documents deleted.


In [2]:
# Check the updated Document
collection.find_one()

{'_id': ObjectId('64453f5a49da71ae65a864ef'),
 'marketplace': 'US',
 'customer_id': 12190288,
 'review_id': 'R3FU16928EP5TC',
 'product_id': 'B00AYB1482',
 'product_parent': 668895143,
 'product_title': 'Enlightened: Season 1',
 'product_category': 'Digital_Video_Download',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'I loved it and I wish there was a season 3',
 'review_body': 'I loved it and I wish there was a season 3... I watched season 2 and loved that as well!',
 'review_date': '2015-08-31'}

## Part 3 - Connect MongoDB with ElasticSearch

In [24]:
!pip install elasticsearch
!pip install elasticsearch_dsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 7.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 8.3 MB/s eta 0:00:0000:01
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 8.7.0
    Uninstalling elasticsearch-8.7.0:
      Successfully uninstalled elasticsearch-8.7.0

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [25]:
!pip install mongo-connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 750.2 kB/s eta 0:00:00 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [44]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Index
from elasticsearch_dsl import Document, Keyword, Text
from mongo_connector import connector, errors
from mongo_connector.connector import Connector

In [45]:
# create Elasticsearch client
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [46]:
# create index mapping
index_mapping = {
    "mappings": {
        "properties": {
            "marketplace": {"type": "keyword"},
            "customer_id": {"type": "long"},
            "review_id": {"type": "keyword"},
            "product_id": {"type": "keyword"},
            "product_parent": {"type": "long"},
            "product_title": {"type": "text"},
            "product_category": {"type": "keyword"},
            "star_rating": {"type": "float"},
            "helpful_votes": {"type": "long"},
            "total_votes": {"type": "long"},
            "vine": {"type": "keyword"},
            "verified_purchase": {"type": "keyword"},
            "review_headline": {"type": "text"},
            "review_body": {"type": "text"},
            "review_date": {"type": "date", "format": "yyyy-MM-dd"}
        }
    }
}

In [ ]:
# Only for DEMO !!!!!!!!!!
# Insert some dummy data to ES Cluster for testing
# create index
index_name = "amazonreview_index"
if not es.indices.exists(index_name):
    es.indices.create(index=index_name, body=index_mapping)

# insert data
doc = {
    "marketplace": None,
    "customer_id": 12190288,
    "review_id": "R3FU16928EP5TC",
    "product_id": "B00AYB1482",
    "product_parent": 668895143,
    "product_title": "Enlightened: Season 1",
    "product_category": "Digital_Video_Download",
    "star_rating": 5,
    "helpful_votes": 0,
    "total_votes": 0,
    "vine": "N",
    "verified_purchase": "Y",
    "review_headline": "I loved it and I wish there was a season 3",
    "review_body": "I loved it and I wish there was a season 3... I watched season 2 and loved that as well!",
    "review_date": "2015-08-31"
}

es.index(index=index_name, body=doc)

In [58]:
# populate elasticsearh index with multithreading 

import concurrent.futures

index_name = "amazonreview_index"
if not es.indices.exists(index_name):
    es.indices.create(index=index_name, body=index_mapping)

def index_doc(doc):
    doc_body = doc.copy()
    doc_body.pop('_id')  # Remove _id field from the body
    es.index(index='amazonreview_index', id=str(doc['_id']), body=doc_body)

with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    futures = []
    for doc in collection.find():
        futures.append(executor.submit(index_doc, doc))

/var/folders/rc/g8bfnkbn2w39f_gxsh1pffc80000gq/T/ipykernel_37700/2299719712.py:3: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if not es.indices.exists(index_name):
/var/folders/rc/g8bfnkbn2w39f_gxsh1pffc80000gq/T/ipykernel_37700/2299719712.py:12: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index='amazonreview_index', id=str(doc['_id']), body=doc_body)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163


KeyboardInterrupt: 

In [59]:
# Display Elasticsearch index populated data for checking purpose

# Define the scroll size and duration
scroll_size = 1000
scroll_duration = '2m'

# Define the query to retrieve the first 10 documents
query = {"query": {"match_all": {}}}
search_results = es.search(index=index_name, body=query, scroll=scroll_duration, size=scroll_size)

# Retrieve the first batch of documents
scroll_id = search_results['_scroll_id']
hits = search_results['hits']['hits']

# Print the first 10 document IDs
for hit in hits[:10]:
    print(hit['_id'])

# Scroll through the remaining documents
while len(hits) > 0:
    search_results = es.scroll(scroll_id=scroll_id, scroll=scroll_duration)
    scroll_id = search_results['_scroll_id']
    hits = search_results['hits']['hits']
    
    # Print the IDs of the next batch of documents
    for hit in hits[:10]:
        print(hit['_id'])

/var/folders/rc/g8bfnkbn2w39f_gxsh1pffc80000gq/T/ipykernel_37700/345334181.py:7: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  search_results = es.search(index=index_name, body=query, scroll=scroll_duration, size=scroll_size)


vv44sIcBjMUX0Fq6ouC4
64453f5a49da71ae65a86593
64453f5a49da71ae65a86594
64453f5a49da71ae65a86595
64453f5a49da71ae65a86596
64453f5a49da71ae65a86597
64453f5a49da71ae65a86598
64453f5a49da71ae65a86599
64453f5a49da71ae65a8659a
64453f5a49da71ae65a8659b
64453f5a49da71ae65a8697a
64453f5a49da71ae65a8697b
64453f5a49da71ae65a8697c
64453f5a49da71ae65a8697d
64453f5a49da71ae65a8697e
64453f5a49da71ae65a8697f
64453f5a49da71ae65a86980
64453f5a49da71ae65a86981
64453f5a49da71ae65a86982
64453f5a49da71ae65a86983
64453f5a49da71ae65a86d62
64453f5a49da71ae65a86d63
64453f5a49da71ae65a86d64
64453f5a49da71ae65a86d65
64453f5a49da71ae65a86d66
64453f5a49da71ae65a86d67
64453f5a49da71ae65a86d68
64453f5a49da71ae65a86d69
64453f5a49da71ae65a86d6a
64453f5a49da71ae65a86d6b
64453f5a49da71ae65a8714a
64453f5a49da71ae65a8714b
64453f5a49da71ae65a8714c
64453f5a49da71ae65a8714d
64453f5a49da71ae65a8714e
64453f5a49da71ae65a8714f
64453f5a49da71ae65a87150
64453f5a49da71ae65a87151
64453f5a49da71ae65a87152
64453f5a49da71ae65a87153
6445

## Part 4 - Display data with Flask Front-end API

### The Flask has three APIs for users to interact with it. The first API is called "/searchProductCountsByRating" which allows users to view the count of products based on their selection on rating from 1-5. The second API is called "/searchVoteCountsByRating" which allows users to view the counts of helpful votes and total votes based on their selection on rating from 1-5. The third API is called "/reviewHeadlinesWithVotes" which allws users to choose view review headlines with or without votes. The .html files are populated with CSS features.

### The main flask .py file is called APAN5400_Proj_Flask.py. To run the code, need to put all the .html files into a folder called "templates"

In [ ]:
# APAN5400_Proj_Flask.py

# 3 Flask Endpoints to query Elasticsearch Index - amazonreview_index

from flask import Flask, render_template, request, jsonify
from elasticsearch import Elasticsearch

app = Flask(__name__)

# Connect to Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/searchProductCountsByRating')
def search_product_counts_by_rating():
    return render_template('selectRating.html', title='Search Product Counts by Rating', field_name='star_rating')

@app.route('/searchVoteCountsByRating')
def search_vote_counts_by_rating():
    return render_template('selectRating.html', title='Search Votes Counts by Rating', field_name='star_rating')


@app.route('/selectRating', methods=['GET', 'POST'])
def selectRating():
    title = request.args.get('title')
    field_name = request.args.get('field_name')
    query = request.args.get('query')

    if title == "Search Product Counts by Rating":
        query = {
            "query": {
                "bool": {
                    "must": [
                        {"match": {field_name: query}},
                        {"range": {"star_rating": {"gte": 1, "lte": 5}}}
                    ]
                }
            },
            "size": 0,
            "aggs": {
                "ratings": {
                    "terms": {"field": "star_rating"}
                }
            }
        }

        result = es.search(index='amazonreview_index', body=query)

        buckets = result['aggregations']['ratings']['buckets']
        rating_counts = {str(bucket['key']): bucket['doc_count'] for bucket in buckets}

        return render_template('productCountsByRatingResults.html', title=title, query=query, results=rating_counts)

    if title == "Search Votes Counts by Rating":
        query = {
            "size": 0,
            "aggs": {
                "rating_stats": {
                    "terms": {"field": field_name},
                    "aggs": {
                        "total_helpful_votes": {"sum": {"field": "helpful_votes"}},
                        "total_votes": {"sum": {"field": "total_votes"}}
                    }
                }
            }
        }

        result = es.search(index='amazonreview_index', body=query)
        rating_stats = result['aggregations']['rating_stats']['buckets']

        votes_by_rating = {}
        for bucket in rating_stats:
            rating = bucket['key']
            helpful_votes = bucket['total_helpful_votes']['value']
            total_votes = bucket['total_votes']['value']
            votes_by_rating[rating] = {'helpful_votes': helpful_votes, 'total_votes': total_votes}

        return render_template('voteCountsByRatingResults.html', title=title, query=query, results=votes_by_rating)

@app.route('/reviewHeadlinesWithVotes', methods=['GET', 'POST'])
def review_headlines_with_votes():
    if request.method == 'POST':
        # Get the user's selection of whether they want to see review headlines with or without votes
        votes_option = request.form['votes_option']

        if votes_option == 'with_votes':
            total_votes_filter = {"gt": 0}  # Only show review headlines with votes (total_votes > 0)
        else:
            total_votes_filter = {"lt": 1}  # Only show review headlines without votes

        query = {
            "query": {
                "range": {
                    "total_votes": total_votes_filter
                }
            },
            "size": 10000,
            "_source": ["review_headline"]
        }

        result = es.search(index='amazonreview_index', body=query)

        review_headlines = [hit['_source']['review_headline'] for hit in result['hits']['hits']]

        return render_template('reviewHeadlines.html', review_headlines=review_headlines, votes_option=votes_option)

    # Render the initial form for the user to select whether they want to see review headlines with or without votes
    return render_template('votesForm.html')



if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# home.html

<!DOCTYPE html>
<html>
  <head>
    <title>Amazon Review Search</title>
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f9f9f9;
      }
      h1 {
        color: #008fd3;
        text-align: center;
        margin-top: 50px;
      }
      h2 {
        color: #555;
        margin-top: 30px;
      }
      form {
        width: 50%;
        margin: auto;
        background-color: #fff;
        padding: 20px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
      }
      label {
        display: block;
        margin-bottom: 10px;
        color: #555;
      }
      select {
        width: 100%;
        padding: 10px;
        border-radius: 5px;
        border: 1px solid #ccc;
        box-sizing: border-box;
      }
      input[type="submit"] {
        display: block;
        margin-top: 20px;
        background-color: #008fd3;
        color: white;
        text-decoration: none;
        padding: 10px 20px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
        border: none;
        cursor: pointer;
      }
      input[type="submit"]:hover {
        background-color: #005f8d;
      }
      #logo {
        width: 200px;
        position: absolute;
        top: 20px;
        left: 20px;
      }
    </style>
  </head>
  <body>
    <img id="logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Amazon_logo.svg/1280px-Amazon_logo.svg.png" alt="Amazon logo">
    <h1>Amazon Review Search</h1>
    <h2>Filter by Votes</h2>
    <form action="/reviewHeadlinesWithVotes" method="POST">
      <label for="vote_filter">Select a vote filter:</label>
      <select name="votes_option" id="vote_filter">
        <option value="with_votes">Reviews with Votes</option>
        <option value="without_votes">Reviews without Votes</option>
      </select>
      <input type="submit" value="Submit">
    </form>
  </body>
</html>



In [ ]:
# selectRating.html

<!DOCTYPE html>
<html>
  <head>
    <title>{{ title }}</title>
    <style>
      /* Global styles */
      body {
        font-family: Arial, sans-serif;
        background-color: #f9f9f9;
      }
      a {
        color: #008fd3;
        text-decoration: none;
      }
      h1 {
        color: #008fd3;
        text-align: center;
        margin-top: 50px;
      }

      /* Form styles */
      form {
        width: 50%;
        margin: auto;
        background-color: #fff;
        padding: 20px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
      }
      label {
        display: block;
        margin-bottom: 10px;
        color: #555;
        font-size: 20px;
      }
      input[type="text"] {
        width: 100%;
        padding: 10px;
        border-radius: 5px;
        border: 1px solid #ccc;
        box-sizing: border-box;
        font-size: 20px;
        margin-bottom: 20px;
      }
      input[type="submit"] {
        display: block;
        margin-top: 20px;
        background-color: #008fd3;
        color: white;
        text-decoration: none;
        padding: 10px 20px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
        border: none;
        cursor: pointer;
        font-size: 20px;
        transition: background-color 0.3s ease;
      }
      input[type="submit"]:hover {
        background-color: #005f8d;
      }

      /* Logo styles */
      #logo {
        width: 200px;
        position: absolute;
        top: 20px;
        left: 20px;
        filter: drop-shadow(2px 2px 5px rgba(0, 0, 0, 0.3));
      }
    </style>
  </head>
  <body>
    <img id="logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Amazon_logo.svg/1280px-Amazon_logo.svg.png" alt="Amazon logo">
    <h1>Search Counts By Rating</h1>
    <form action="/selectRating" method="GET">
      <label for="query">Enter ratings from 1-5 :</label>
      <input type="text" id="query" name="query">
      <input type="hidden" name="title" value="{{ title }}">
      <input type="hidden" name="field_name" value="{{ field_name }}">
      <input type="submit" value="Submit">
    </form>
  </body>
</html>



In [ ]:
# productCountsByRatingResults.html

<!DOCTYPE html>
<html>
  <head>
    <title>Search Results</title>
    <style>
      /* add CSS styles here */
      body {
        margin: 0;
        padding: 0;
        font-family: Arial, sans-serif;
      }
      header {
        background-color: #232f3e;
        color: white;
        display: flex;
        align-items: center;
        padding: 10px;
      }
      header img {
        height: 50px;
        margin-right: 10px;
      }
      h1 {
        margin-top: 30px;
        text-align: center;
        font-size: 36px;
      }
      table {
        margin: 0 auto;
        border-collapse: collapse;
        border: 2px solid #232f3e;
        width: 50%;
        margin-top: 30px;
      }
      th, td {
        padding: 10px;
        text-align: center;
        border: 1px solid #232f3e;
      }
      th {
        background-color: #232f3e;
        color: white;
      }
      tr:nth-child(even) {
        background-color: #f2f2f2;
      }
      a {
        display: block;
        margin-top: 20px;
        text-align: center;
        color: #232f3e;
        text-decoration: none;
      }
      #logo {
        width: 200px;
        position: absolute;
        top: 20px;
        left: 20px;
        filter: drop-shadow(2px 2px 5px rgba(0, 0, 0, 0.3));
      }
    </style>
  </head>
  <body>
    <header>
      <img src="https://pngimg.com/uploads/amazon/amazon_PNG11.png" alt="Amazon Logo">
      <h1>Search Results</h1>
    </header>
    <table>
      <thead>
        <tr>
          <th>Rating</th>
          <th>Product Count</th>
        </tr>
      </thead>
      <tbody>
        {% for rating, count in results.items() %}
        <tr>
          <td>{{ rating }}</td>
          <td>{{ count }}</td>
        </tr>
        {% endfor %}
      </tbody>
    </table>
    <a href="/">Back to Search</a>
  </body>
</html>


In [ ]:
# voteCountsByRatingResults.html

<!DOCTYPE html>
<html>
  <head>
    <title>Search Results</title>
    <style>
      /* add CSS styles here */
      body {
        margin: 0;
        padding: 0;
        font-family: Arial, sans-serif;
      }
      header {
        background-color: #232f3e;
        color: white;
        display: flex;
        align-items: center;
        padding: 10px;
      }
      header img {
        height: 50px;
        margin-right: 10px;
      }
      h1 {
        margin-top: 30px;
        text-align: center;
        font-size: 36px;
      }
      table {
        margin: 0 auto;
        border-collapse: collapse;
        border: 2px solid #232f3e;
        width: 50%;
        margin-top: 30px;
      }
      th, td {
        padding: 10px;
        text-align: center;
        border: 1px solid #232f3e;
      }
      th {
        background-color: #232f3e;
        color: white;
      }
      tr:nth-child(even) {
        background-color: #f2f2f2;
      }
      a {
        display: block;
        margin-top: 20px;
        text-align: center;
        color: #232f3e;
        text-decoration: none;
      }
      #logo {
        width: 200px;
        position: absolute;
        top: 20px;
        left: 20px;
        filter: drop-shadow(2px 2px 5px rgba(0, 0, 0, 0.3));
      }

    </style>
  </head>
  <body>
    <img id="logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Amazon_logo.svg/1280px-Amazon_logo.svg.png" alt="Amazon logo">
    <h1>Search Results</h1>
    <table>
      <thead>
        <tr>
          <th>Rating</th>
          <th>Total Helpful Votes Count</th>
          <th>Total Votes Count</th>
        </tr>
      </thead>
      <tbody>
        {% for rating, votes in results.items() %}
        <tr>
          <td>{{ rating }}</td>
          <td>{{ votes['helpful_votes'] }}</td>
          <td>{{ votes['total_votes'] }}</td>
        </tr>
        {% endfor %}
      </tbody>
    </table>
    <a href="/">Back to Search</a>
  </body>
</html>


In [ ]:
# votesForm.html

<!DOCTYPE html>
<html>
  <head>
    <title>Amazon Review Search</title>
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f9f9f9;
      }
      h1 {
        color: #008fd3;
        text-align: center;
        margin-top: 50px;
      }
      h2 {
        color: #555;
        margin-top: 30px;
      }
      form {
        width: 50%;
        margin: auto;
        background-color: #fff;
        padding: 20px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
      }
      label {
        display: block;
        margin-bottom: 10px;
        color: #555;
      }
      select {
        width: 100%;
        padding: 10px;
        border-radius: 5px;
        border: 1px solid #ccc;
        box-sizing: border-box;
      }
      input[type="submit"] {
        display: block;
        margin-top: 20px;
        background-color: #008fd3;
        color: white;
        text-decoration: none;
        padding: 10px 20px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
        border: none;
        cursor: pointer;
      }
      input[type="submit"]:hover {
        background-color: #005f8d;
      }
      #logo {
        width: 200px;
        position: absolute;
        top: 20px;
        left: 20px;
      }
    </style>
  </head>
  <body>
    <img id="logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Amazon_logo.svg/1280px-Amazon_logo.svg.png" alt="Amazon logo">
    <h1>Amazon Review Search</h1>
    <h2>Filter by Votes</h2>
    <form action="/reviewHeadlinesWithVotes" method="POST">
      <label for="vote_filter">Select a vote filter:</label>
      <select name="votes_option" id="vote_filter">
        <option value="with_votes">Reviews with Votes</option>
        <option value="without_votes">Reviews without Votes</option>
      </select>
      <input type="submit" value="Submit">
    </form>
  </body>
</html>




In [ ]:
# reviewHeadlines.html

<!DOCTYPE html>
<html>
  <head>
    <title>Amazon Review Search Results</title>
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f9f9f9;
      }
      h1 {
        color: #008fd3;
        text-align: center;
        margin-top: 50px;
      }
      p {
        color: #555;
        text-align: center;
        font-size: 1.2rem;
        margin-top: 30px;
      }
      ul {
        list-style: none;
        margin: 0;
        padding: 0;
        text-align: center;
      }
      li {
        background-color: #fff;
        padding: 10px;
        margin-bottom: 10px;
        border-radius: 5px;
        box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.3);
        color: #333;
      }
      a {
        display: block;
        text-align: center;
        margin-top: 20px;
        color: #008fd3;
      }
      #logo {
        width: 200px;
        position: absolute;
        top: 20px;
        left: 20px;
      }
    </style>
  </head>
  <body>
    <img id="logo" src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Amazon_logo.svg/1280px-Amazon_logo.svg.png" alt="Amazon logo">
    <h1>Amazon Review Search Results</h1>
    <p>You are viewing {{ votes_option }} review headlines:</p>
    <ul>
      {% for headline in review_headlines %}
        <li>{{ headline }}</li>
      {% endfor %}
    </ul>
    <a href="/reviewHeadlinesWithVotes">Back to Filter</a>
  </body>
</html>

